In [25]:
#Notebook description

#this notebook is used to backtest the performance of multiple mechanical trading strategies on a given stock to see how the asset should be
#effectively traded.

In [26]:
#define custom functions to be used in the notebook
def simplify_datetime_index(series):
    """
    Simplifies the DateTime index of a Series to contain only the date (YYYY-MM-DD),
    maintaining it as a DateTimeIndex without timezone information.
    
    Parameters:
        series (pd.Series): The input Series with a DateTimeIndex.
    
    Returns:
        pd.Series: The Series with the DateTime index simplified to YYYY-MM-DD.
    """
    if not isinstance(series.index, pd.DatetimeIndex):
        raise TypeError("The Series index must be a DateTimeIndex.")
    
    # Remove timezone information if present
    if series.index.tz is not None:
        series = series.copy()
        series.index = series.index.tz_convert('UTC').tz_localize(None)
    
    # Normalize the index to remove the time component
    series.index = series.index.normalize()
    
    return series

In [27]:
#load the necessary libraries and paramets
import vectorbt as vbt
import yfinance as yf
import pandas as pd

time_frame_week = 7
time_frame_short = 21
time_frame_mid   = 50
time_frame_long = 200
interval = '1d'
period     = '10y'

ticker_str = 'XLV'
#download the data
ticker = yf.Ticker(ticker_str).history(period=period, interval=interval)
benchmark = yf.Ticker('SPY').history(period=period, interval=interval)

ticker = simplify_datetime_index(ticker)
benchmark = simplify_datetime_index(benchmark)
ticker = ticker[ticker.index.isin(benchmark.index)]

In [28]:
#Define strategies

def buy_and_hold_strategy(close_prices):
    """
    Deploys a buy and hold strategy using vectorbt.

    Parameters:
        close_prices (pd.Series): Series of closing prices for the ticker.

    Returns:
        vbt.Portfolio: The portfolio object containing backtest results.
    """
    # Define entry signal: Buy at the first available date
    entries = pd.Series(False, index=close_prices.index)
    entries.iloc[0] = True

    # Define exit signal: Sell at the last available date
    exits = pd.Series(False, index=close_prices.index)
    exits.iloc[-1] = True

    # Create the portfolio
    portfolio = vbt.Portfolio.from_signals(
        close=close_prices,
        entries=entries,
        exits=exits,
        init_cash=10000,     # Initial capital
        fees=0.001,          # Transaction fees (0.1%)
        freq='D'             # Frequency of data ('D' for daily)
    )

    return portfolio


In [ ]:
# Example usage:
portfolio = buy_and_hold_strategy(ticker['Close'])

# Print portfolio statistics
print(portfolio.stats())

# Plot the portfolio's total value over time
portfolio.value().vbt.plot(title='Buy and Hold Strategy Total Value').show()

# Compare with the benchmark (SPY)
benchmark_returns = benchmark['Close'].pct_change().fillna(0)
benchmark_cum_returns = (1 + benchmark_returns).cumprod()

# Plot both portfolio and benchmark
import plotly.graph_objects as go

fig = go.Figure()

# Portfolio value normalized to start at 1
portfolio_normalized = portfolio.value() / portfolio.value().iloc[0]
fig.add_trace(go.Scatter(
    x=portfolio_normalized.index,
    y=portfolio_normalized.values,
    mode='lines',
    name='Buy and Hold Strategy'
))

# Benchmark cumulative returns normalized to start at 1
benchmark_normalized = benchmark_cum_returns / benchmark_cum_returns.iloc[0]
fig.add_trace(go.Scatter(
    x=benchmark_normalized.index,
    y=benchmark_normalized.values,
    mode='lines',
    name='Benchmark (SPY)'
))

fig.update_layout(
    title='Buy and Hold Strategy vs Benchmark',
    xaxis_title='Date',
    yaxis_title='Normalized Value',
    legend=dict(x=0, y=1),
    hovermode='x unified'
)

fig.show()